### 图像轮廓
- 在轮廓检测前必须进行二值化处理
- cv2.findContours(img,mode,method)
1) mode只需记住最后一个参数RETR_TREE, 其它结果都可从中调用
2) method即轮廓的逼近方法<br>
CHAIN_APPROX_NONE: 输出多边形<br>
CHAIN_APPROX_SIMPLE: 只保留终点部分


In [8]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [149]:
def cv_show(img, title='Image'):
    cv2.imshow(title, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [150]:
img= cv2.imread('shape.png', cv2.IMREAD_GRAYSCALE)

In [151]:
img_color= cv2.imread('shape.png')

In [152]:
# 二值化
thresh, ret= cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

In [153]:
# 寻找轮廓
contours, hierarchy= cv2.findContours(ret, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

In [142]:
# 画轮廓，-1 means draw all contours, 2 means thickness of the contour lines
res= cv2.drawContours(img_color, contours, 0, (0, 0, 255), 2)
cv_show(res, "res")

In [144]:
# 提取轮廓特征
cnt= contours[0]   # 选择第一个轮廓

In [145]:
cv2.contourArea(cnt)  # 计算轮廓面积

82533.5

In [146]:
cv2.arcLength(cnt, True)  # 计算轮廓周长, True表示轮廓是闭合的

1843.1097334623337

In [ ]:
# 轮廓近似
epsilon= 0.01*cv2.arcLength(cnt, True) # 是否需要将直线二分的阈值, epsilon越小，则近似越精确
approx= cv2.approxPolyDP(cnt, epsilon, True) 
draw_img= img_color.copy()
res= cv2.drawContours(draw_img, [approx], -1, (0, 255, 0), 2)  # 绘制近似轮廓
cv_show(res, "approx")

In [161]:
# 轮廓的外接矩形
x, y, w, h = cv2.boundingRect(cnt)  # 获取外接矩形的坐标和宽高
img= cv2.rectangle(img_color, (x, y), (x + w, y + h), (255, 0, 0), 2)  # 绘制外接矩形
cv_show(img, "boundingRect")

In [162]:
# 轮廓的外接圆
(x,y),radius = cv2.minEnclosingCircle(cnt)  
center= (int(x), int(y))
radius= int(radius)
img= cv2.circle(img_color, center, radius, (0, 255, 255), 2)  # 绘制外接圆
cv_show(img, "minEnclosingCircle")

### 模板匹配
#### 模板匹配类似于卷积，模板在原图像上从原点开始滑动，计算模板与原图像中被覆盖区域的差别程度，这个计算方法有6种。假设原图像是A*B，而模板是a*b大小，则输出结果的矩阵是(A-a+1)*(B-b+1)
- TM_SQDIFF_NORMED, 正规化后的diff, 越接近于0越匹配
- TM_CCORR_NORMED，正规化后的相关系数，越接近于1越匹配<br>
#### matchTemplate返回的结果是各个移动框所对应的差别程度，可用cv2.minMaxLoc来解析这个结果
#### 不适用于模板和比较图像是不同尺度的情况

In [58]:

lena_face= cv2.imread('lena_face.png', cv2.IMREAD_GRAYSCALE)
lena= cv2.imread('lenanoise.png', cv2.IMREAD_GRAYSCALE)
h, w= lena_face.shape

In [59]:
res= cv2.matchTemplate(lena, lena_face, cv2.TM_SQDIFF_NORMED)

In [60]:
res.shape

(311, 320)

In [61]:
# min_loc为移动框左上角的坐标
min_val, max_val, min_loc, max_loc= cv2.minMaxLoc(res)

In [62]:
print(min_val, min_loc)

0.01317409798502922 (184, 198)


In [ ]:
top_left= min_loc 
bottm_right= (top_left[0] + w, top_left[1] + h) # 因为在tangle里面，起始点是左上角，而终点在右下角，所以是加上h

In [64]:
cv2.rectangle(lena, min_loc, bottm_right, (0, 0, 255), 2)

array([[253, 254, 255, ..., 248, 249, 114],
       [253, 253, 254, ..., 248, 249, 113],
       [252, 252, 253, ..., 249, 249, 112],
       ...,
       [ 49,  50,  50, ...,  88,  95,  99],
       [ 59,  61, 234, ...,  91,  97, 100],
       [ 49, 233,  54, ...,  96, 100, 104]], shape=(537, 512), dtype=uint8)

In [65]:
def cv_show(img, title='Image'):
    cv2.imshow(title, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [69]:
cv_show(lena, "lena")

In [ ]:
# 多个模板匹配
array= np.array([[0, 0], [10, 10], [20, 20], [30, 30]])
res= np.where(array> 10)  # 返回满足条件的索引
list(zip(*res)) # *res将返回的元组中的element解放出来，zip将这些element中的value根据position不同regroup


[(np.int64(2), np.int64(0)),
 (np.int64(2), np.int64(1)),
 (np.int64(3), np.int64(0)),
 (np.int64(3), np.int64(1))]

In [90]:
res

(array([2, 2, 3, 3]), array([0, 1, 0, 1]))

### 图像金字塔
- 高斯金字塔
1) 向下采样(缩小)
1) 向上采样
- 拉普拉斯金字塔<br>
Li= Gi-PyrUp(PyrDown(Gi))

In [73]:
img= cv2.imread('cat.jpg', cv2.IMREAD_GRAYSCALE)

In [74]:
cv_show(img)

In [75]:
up= cv2.pyrUp(img)

In [76]:
cv_show(up)

In [78]:
# 拉普拉斯金字塔
down= cv2.pyrDown(img)
down_up= cv2.pyrUp(down)
l_1= img-down_up
cv_show(l_1, "l_1")